Price history

In [31]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from PIL import Image
import requests
import datetime
import re
import time
import urllib
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np

url = "https://www.larvalabs.com/cryptopunks/details/1119"

In [32]:
def load_punk_info (url):
    web_r = requests.get(url)
    websoup = BeautifulSoup(web_r.text, "html.parser")
    
    price_info = pd.DataFrame()
    for titlesoup in websoup.findAll("div",{"class":"table-responsive"}):
        for element in titlesoup.findAll("tr", {"class":re.compile(r"\bpunk-history-row\b")}):
            price_info= price_info.append([element.get_text().replace("\n", "")])
    return price_info
    
def transform_punk_info (price_info):   
    price_info = price_info[0].str.split(expand=True)
    price_info["Bid"] = np.where(price_info[0].str.contains("Bid"), 1, 0)
    price_info["Offered"] = np.where(price_info[0].str.contains("Offered"), 1, 0)
    price_info["Transfer"] = np.where(price_info[0].str.contains("Transfer"), 1, 0)
    price_info["Claimed"] = np.where(price_info[0].str.contains("Claimed"), 1, 0)
    price_info["Withdrawn"] = np.where(price_info[1].str.contains("Withdrawn"), 1, 0)
    price_info["Offer"] = np.where(price_info[0].str.contains("Offer"), 1, 0)
    
    price_info["price_in_dollars"] = price_info[1].apply(lambda x: re.findall(r"\(([A-Za-z0-9_$,.<]+)\)", x))
    price_info["price_in_dollars"] = np.where(price_info.Withdrawn == 1, price_info[2].apply(lambda x: re.findall(r"\(([A-Za-z0-9_$,.<]+)\)", x)), price_info["price_in_dollars"])
    price_info = price_info.explode("price_in_dollars")
    #print(price_info.head())
    price_info["Year"] = price_info[4].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x))
    price_info = price_info.explode("Year")
    
    price_info["Year"] = np.where(price_info.Year.isnull(), price_info[3].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x)), price_info["Year"])
    price_info = price_info.explode("Year")
    
    price_info["Year"] = np.where(price_info.Year.isnull(), price_info[2].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x)), price_info["Year"])
    price_info = price_info.explode("Year")
    
    price_info["Month"] = price_info[1].astype(str).apply(lambda x: re.findall(r"\b[a-zA-Z]{3}\b", x))
    price_info = price_info.explode("Month")
    
    price_info["Month"] = np.where(price_info.Month.isnull(), price_info[2].astype(str).apply(lambda x: re.findall(r"\b[a-zA-Z]{3}\b", x)), price_info["Month"])
    price_info = price_info.explode("Month")
    price_info = price_info.iloc[:,6:]
    return price_info

In [33]:
price_info = load_punk_info(url)

In [34]:
price_info = transform_punk_info(price_info)

Iterative loop

In [35]:
path = r"C:\Users\Acer\Git\deda_punks\Crypto_punks\bids"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
punks = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            punks.append(file.name)

In [36]:
import itertools
import re
new_punks = [re.findall("(\d+)", element) for element in punks]
new_punks = list(itertools.chain(*new_punks))

In [161]:
import tqdm
import time

final_df = pd.DataFrame()
for i in tqdm.tqdm(new_punks[560:590]):
    try:
        url = "https://www.larvalabs.com/cryptopunks/details/" + i
        price_info = load_punk_info(url)
        price_info = transform_punk_info(price_info)
        price_info["Punk_ID"] = i
        final_df = final_df.append(price_info)
        time.sleep(3)
    except:
        print("Fuck!", i)

  3%|██▊                                                                                | 1/30 [00:11<05:40, 11.74s/it]

Fuck! 6370


 10%|████████▎                                                                          | 3/30 [00:15<01:49,  4.07s/it]

Fuck! 6375


 13%|███████████                                                                        | 4/30 [00:16<01:12,  2.79s/it]

...

Fuck! 6687


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [00:59<00:00,  1.53it/s]

Fuck! 6691


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [01:00<00:00,  2.02s/it]

Fuck! 6714


In [143]:
final_df.Punk_ID.nunique()

10

In [84]:
len(new_punks)

865

In [85]:
[new_punks.remove(element) for element in final_df.Punk_ID.unique()]

ValueError: list.remove(x): x not in list

In [59]:
final_df.to_csv(r"Crypto_Punks_with_prices.csv", index = None)

In [70]:
final_df.to_csv(r"Crypto_Punks_with_prices2.csv", index = None)

In [76]:
final_df.to_csv(r"Crypto_Punks_with_prices3.csv", index = None)

In [79]:
final_df.to_csv(r"Crypto_Punks_with_prices4.csv", index = None)

In [86]:
final_df.to_csv(r"Crypto_Punks_with_prices5.csv", index = None)

In [89]:
final_df.to_csv(r"Crypto_Punks_with_prices6.csv", index = None)

In [92]:
final_df.to_csv(r"Crypto_Punks_with_prices7.csv", index = None)

In [95]:
final_df.to_csv(r"Crypto_Punks_with_prices8.csv", index = None)

In [98]:
final_df.to_csv(r"Crypto_Punks_with_prices9.csv", index = None)

In [101]:
final_df.to_csv(r"Crypto_Punks_with_prices10.csv", index = None)

In [114]:
pd.read_csv(r"C:\Users\Acer\Git\deda_punks\Crypto_Punks_with_prices1.csv")

,Offered,Transfer,Claimed,Withdrawn,Offer,price_in_dollars,Year,Month,Punk_ID
0,0,0,0,1,0,"$68,985",2021,Apr,8
1,0,0,0,0,0,"$62,427",2021,Apr,8
2,0,0,0,1,0,"$41,618",2021,Apr,8
3,0,0,0,0,0,"$40,949",2021,Apr,8
4,0,0,0,0,0,"$3,618",2021,Mar,8
...,...,...,...,...,...,...,...,...,...
95,0,0,1,0,0,NaN,2017,NaN,336
96,0,0,0,1,0,"$1,074",2021,Apr,363
97,0,0,0,0,0,$990,2021,Apr,363
98,0,0,0,0,0,$13,2017,Jun,363


In [113]:
os.getcwd()

'C:\\Users\\Acer\\Git\\deda_punks\\Crypto_punks\\bids'

In [115]:
finalfinal = pd.DataFrame()
for i in list(range(1,11)):
    df = pd.read_csv(r"C:\Users\Acer\Git\deda_punks\Crypto_Punks_with_prices"+str(i)+".csv")
    finalfinal = finalfinal.append(df)

In [103]:
list(range(1,11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [159]:
finalfinal = finalfinal.drop_duplicates()

In [121]:
finalfinal.Punk_ID.nunique()

116

In [162]:
finalfinal = finalfinal.append(final_df)

In [160]:
finalfinal.to_csv(r"punks_prices.csv", index = None)

In [156]:
finalfinal.Punk_ID.nunique()

204